# Business Analysis Project Using SQL

We'll be working with a modified version of a database called [Chinook](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.  Here's a [schema diagram for the Chinook database:](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg) 


In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of database schema

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting New Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we have been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name| Genre |
| --- | --- | 
| Regal | Hip-Hop |
| Red Tone | Punk | 
| Meteor and the Girls | Pop | 
| Slim Jim Bites | Blues | 

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

- Subquery - to get all the USA based customers with invoice details
- Main query - to compute percentage of tracks_sold in each genre 



In [3]:

%%sql
WITH usa_tracks_sold AS 
    (SELECT * 
     FROM customer c
     INNER JOIN invoice i ON i.customer_id=c.customer_id
     INNER JOIN invoice_line il ON i.invoice_id=il.invoice_id WHERE c.country="USA")

SELECT
    g.name genre_name,
    SUM(uts.quantity) tracks_sold,
    ROUND (CAST(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) From usa_tracks_sold), 2) AS percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t ON uts.track_id=t.track_id 
INNER JOIN genre g ON g.genre_id=t.genre_id GROUP BY 1 ORDER BY 2 DESC;       



 * sqlite:///chinook.db
Done.


genre_name,tracks_sold,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


**REMARK:** As we examine the popularity of genre based on records sold, Punk, Blue, and Pop are the genres that should be added to the record label.

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing the employee performance 

In [4]:
%%sql

WITH employee_sales AS 
    (SELECT e.employee_id, 
            e.first_name|| " "||e.last_name employee_name,
            e.hire_date,
            c.customer_id, i.invoice_id,i.total 
    FROM employee e 
    INNER JOIN customer c ON e.employee_id=c.support_rep_id 
    INNER JOIN invoice i ON c.customer_id=i.customer_id)
    
SELECT es.employee_name,
       es.hire_date,
       SUM(es.total) total_sales,
       ROUND((CAST(SUM(es.total) AS FLOAT)/(SELECT SUM(total) FROM employee_sales))*100,2) percentage_sales
FROM employee_sales es GROUP BY 1 ORDER BY 3 DESC;

 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_sales,percentage_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039,36.77
Margaret Park,2017-05-03 00:00:00,1584.0000000000034,33.63
Steve Johnson,2017-10-17 00:00:00,1393.920000000002,29.6


**REMARK** Jane has 2% sales more than Park. This could be because Jane was hired one month prior to Park.

## Analyzing Sales by Country

There are a number of countries with only one customer, we will group these customers as "other" in our analysis. 

%%sql
WITH country_or_other AS
     (SELECT 
            CASE
                WHEN (SELECT COUNT(*) FROM customer WHERE country=c.country)=1 THEN "other"
                ELSE country
            END country,
            customer_id
     FROM customer c)
    
SELECT  c.country,
        COUNT(distinct c.customer_id) total_customers,
        ROUND(SUM(i.total),1) total_sales, 
        ROUND(CAST(SUM(i.total) AS FLOAT)/COUNT(c.customer_id),2) avg_sales_per_customer, 
        ROUND(CAST(SUM(i.total) AS FLOAT)/COUNT(i.invoice_id),2) avg_order_value,
        CASE
            WHEN country = "other" THEN 1
            ELSE 0
        END AS sort
FROM country_or_other c 
LEFT JOIN invoice i ON c.customer_id=i.customer_id 
GROUP BY 1 
ORDER BY sort ASC, 4 DESC;

**REMARK**  Based on our analysis, Czech Republic, UK and India are top 3 countries with highest average sales per customer. There is a potential to tap the market in these countries.

## Analyzing percentage of purchases of individual tracks vs whole albums

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

- **Question:** So, we find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

- For performing an analysis we have 'edge cases' which prevent us from getting a 100% accurate answer to our question. In this instance, we have two edge cases to consider:

    - Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
    - Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.
    
Since our analysis is concerned with maximizing revenue we can safely ignore edge cases. 

**Solution: We start by analyzing by identifying whether each invoice has all the tracks from an album.**
- Comparing list of tracks from invoice with list of tracks from an album
    - Find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.
    - NOTE: Album has list of tracks, invoice has list of invoice_line_ids, and invoice_line_id has list of tracks

In [5]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674



Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

## Which artist is used in the most playlists?

In [6]:
%%sql

WITH artist_track AS 
                (SELECT a.name,t.track_id FROM artist a 
                 LEFT JOIN album al ON a.artist_id=al.artist_id 
                 LEFT JOIN track t ON t.album_id=al.album_id ORDER BY 1),
     playlist_name_track AS 
                      (SELECT p.name playlist_name, pt.track_id 
                      FROM playlist_track pt 
                      LEFT JOIN playlist p 
                      ON p.playlist_id=pt.playlist_id 
                      ORDER BY 1) 
    
SELECT at.name, 
       COUNT(pnt.playlist_name) AS in_no_of_playlists,
       ROUND(CAST(COUNT(pnt.playlist_name) AS FLOAT)/ (SELECT COUNT(*) FROM playlist),2) percentage FROM artist_track at 
LEFT JOIN playlist_name_track pnt ON at.track_id=pnt.track_id GROUP BY 1 ORDER BY 2 DESC LIMIT 10;

 * sqlite:///chinook.db
Done.


name,in_no_of_playlists,percentage
Iron Maiden,516,28.67
U2,333,18.5
Metallica,296,16.44
Led Zeppelin,252,14.0
Deep Purple,226,12.56
Lost,184,10.22
Pearl Jam,177,9.83
Eric Clapton,145,8.06
Faith No More,145,8.06
Lenny Kravitz,143,7.94


**REMARK:** Iron Maiden, U2, Metallica, Led Zeppelin and Deep Purple who are listed among top 5 popular artists.

## How many tracks have been purchased vs not purchased?

In [7]:
%%sql
WITH album_track AS
    (
     SELECT
         a.album_id album_id,
         t.track_id track_id
     FROM album a 
     LEFT JOIN track t ON t.album_id=a.album_id
    )

SELECT
    COUNT(DISTINCT(il.track_id)) tracks_purchased,
    (COUNT(DISTINCT(at.track_id)) - COUNT(DISTINCT(il.track_id))) tracks_not_purchased
FROM album_track at 
LEFT JOIN invoice_line il ON at.track_id=il.track_id;

 * sqlite:///chinook.db
Done.


tracks_purchased,tracks_not_purchased
1806,1697


## Is the range of tracks in the store reflective of their sales popularity?

In [8]:
%%sql
SELECT (SELECT COUNT(track_id) FROM track) range_of_tracks,
       COUNT(DISTINCT(track_id)) distinct_tracks_sold, 
       COUNT(*) total_tracks_sold,
       ROUND(CAST(COUNT(DISTINCT(track_id)) AS FLOAT) /(SELECT COUNT(track_id) FROM track),2)*100 "distinct_percentage_of_tracks_sold(%)"
FROM invoice_line;

 * sqlite:///chinook.db
Done.


range_of_tracks,distinct_tracks_sold,total_tracks_sold,distinct_percentage_of_tracks_sold(%)
3503,1806,4757,52.0


Over 50% of the tracks sold from the avaiable tracks. Since, total tracks sold is almost 3 times more than the distinct tracks sold infers that some of the tracks are very popular and we sold more than 1 time.

## Do protected vs non-protected media types have an effect on popularity?

In [9]:
%%sql
SELECT mt.name, 
       COUNT(mt.media_type_id) total_sales
FROM media_type mt
INNER JOIN track t ON mt.media_type_id=t.media_type_id
INNER JOIN invoice_line il ON t.track_id=il.track_id
GROUP BY 1 
ORDER BY 2 DESC;
       

 * sqlite:///chinook.db
Done.


name,total_sales
MPEG audio file,4259
Protected AAC audio file,439
Purchased AAC audio file,35
AAC audio file,21
Protected MPEG-4 video file,3


**REMARK** Protected media type is less popular when compared to unproected media. It could be because copyright restriction. 